In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [139]:
#first step data input and inspecting
ad_clicks = pd.read_csv('ad_clicks.csv')
display(ad_clicks.head())

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


In [142]:
#which platform was getting most view or advertisement range
most_view = ad_clicks.groupby('utm_source').user_id.count().reset_index()
most_view.sort_values(by='user_id', ascending = False)

,utm_source,user_id
2,google,680
1,facebook,504
0,email,255
3,twitter,215


In [180]:
# check if users is actually clicking by inspecting timestamp

#adding new column 'is_clicks'
ad_clicks['is_clicks'] = ~ ad_clicks.ad_click_timestamp.isnull()
# ~ operator express not
display(ad_clicks.head())

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_clicks
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


In [166]:
#we want to know how many percent people actually clicks from a source
clicks_by_source = ad_clicks.groupby(['utm_source','is_clicks']).user_id.count().reset_index()

#pivoting to achieve better understanding
cbs_pivot = clicks_by_source.pivot(columns = 'is_clicks',index='utm_source',values='user_id').reset_index()
csb_pivot_sorted = cbs_pivot.sort_values(by=False,ascending=False).reset_index(drop=True)


#creating percentage column
csb_pivot_sorted['percentage'] = round(csb_pivot_sorted[True]/ (csb_pivot_sorted[False] +csb_pivot_sorted[True]) *100 ,2)
display(csb_pivot_sorted)

is_clicks,utm_source,False,True,percentage
0,google,441,239,35.15
1,facebook,324,180,35.71
2,email,175,80,31.37
3,twitter,149,66,30.70


In [174]:
#analyzing A/B testing

#inspecting test group contestant
ab_testing = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
#display(ab_testing)

#seeking which test percentage is greater from user clicks
ad_percentage = ad_clicks.groupby(['experimental_group','is_clicks']).user_id.count().reset_index()
ad_percentage_pivot = ad_percentage.pivot(columns = 'is_clicks',index='experimental_group',values = 'user_id').reset_index()
ad_percentage_pivot['percentage']= round(ad_percentage_pivot[True] / (ad_percentage_pivot[True] +ad_percentage_pivot[False]) *100,2)

display(ad_percentage_pivot)
#in general, ads A is better than ads B 

is_clicks,experimental_group,False,True,percentage
0,A,517,310,37.48
1,B,572,255,30.83


In [191]:
#get detailed on respectively ads a and b

a_clicks = ad_clicks[ad_clicks.experimental_group.isin(['A'])].reset_index(drop=True)
b_clicks = ad_clicks[ad_clicks['experimental_group'] == 'B'].reset_index(drop =True)

display(a_clicks.head())
display(b_clicks.head())

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_clicks
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
2,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
3,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
4,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False


,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_clicks
0,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
1,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
2,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
3,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B,True
4,01adb2e7-f711-4ae4-a7c6-29f48457eea1,google,3 - Wednesday,NaN,B,False


In [223]:
#calculating users who click on ads by day
click_by_day= a_clicks.groupby(['day','is_clicks']).user_id.count().reset_index()
click_by_day_pvt = click_by_day.pivot(columns = 'is_clicks',index='day',values='user_id').reset_index()
click_by_day_pvt['percentage'] = round((click_by_day_pvt[True] / (click_by_day_pvt[False] +click_by_day_pvt[True]) *100),2)
display(click_by_day_pvt)

click_by_day_b = b_clicks.groupby(['is_clicks','day']).user_id.count().reset_index()
click_by_day_b_pvt = click_by_day_b.pivot(columns = 'is_clicks',index = 'day',values='user_id').reset_index()
click_by_day_b_pvt['percentage'] = round((click_by_day_b_pvt[True] / (click_by_day_b_pvt[True] +click_by_day_b_pvt[False]) *100),2)
display(click_by_day_b_pvt)

is_clicks,day,False,True,percentage
0,1 - Monday,70,43,38.05
1,2 - Tuesday,76,43,36.13
2,3 - Wednesday,86,38,30.65
3,4 - Thursday,69,47,40.52
4,5 - Friday,77,51,39.84
5,6 - Saturday,73,45,38.14
6,7 - Sunday,66,43,39.45


is_clicks,day,False,True,percentage
0,1 - Monday,81,32,28.32
1,2 - Tuesday,74,45,37.82
2,3 - Wednesday,89,35,28.23
3,4 - Thursday,87,29,25.00
4,5 - Friday,90,38,29.69
5,6 - Saturday,76,42,35.59
6,7 - Sunday,75,34,31.19


In [229]:
#detailed source user count and its correlation with day
insight_a = a_clicks.groupby(['utm_source','day','is_clicks']).user_id.count().reset_index()
insight_a_true = insight_a[insight_a.is_clicks == True].reset_index()
insight_a_true_pvt = insight_a_true.pivot(columns = 'utm_source',index='day',values = 'user_id').reset_index()
display(insight_a_true_pvt)

insight_b = b_clicks.groupby(['utm_source','day','is_clicks']).user_id.count().reset_index()
insight_b_true = insight_b[insight_b.is_clicks == True].reset_index()
insight_b_true_pvt = insight_b_true.pivot(columns = 'utm_source',index='day',values='user_id').reset_index()
display(insight_b_true_pvt)

utm_source,day,email,facebook,google,twitter
0,1 - Monday,10,15,17,1
1,2 - Tuesday,7,13,19,4
2,3 - Wednesday,7,12,13,6
3,4 - Thursday,2,13,25,7
4,5 - Friday,4,14,26,7
5,6 - Saturday,5,14,19,7
6,7 - Sunday,6,13,18,6


utm_source,day,email,facebook,google,twitter
0,1 - Monday,4,12,13,3
1,2 - Tuesday,4,15,20,6
2,3 - Wednesday,7,7,16,5
3,4 - Thursday,6,11,7,5
4,5 - Friday,4,14,17,3
5,6 - Saturday,4,13,20,5
6,7 - Sunday,10,14,9,1
